In [2]:
import os
import yaml
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

config_path = "config.yml"
with open(config_path) as f:
    config = yaml.safe_load(f)
    
email = config["username"]
password = config["password"]
subject = config["subject"]
data = []

s = Service("path_to_chromedriver")
driver = webdriver.Chrome(service=s)
driver.get('https://in.linkedin.com/?src=go-pa&trk=sem-ga_campid.14650114788_asid.151761418307_crid.657403558721_kw.linkedin+login_d.c_tid.kwd-12704335873_n.g_mt.e_geo.9298387&mcid=6844056167778418689&cid=&gclid=EAIaIQobChMIs4n23IOUgAMVyA97Bx3e5QNFEAAYASAAEgKQf_D_BwE&gclsrc=aw.ds&original_referer=https%3A%2F%2Fwww.google.com%2F')
driver.maximize_window()

wait = WebDriverWait(driver, 10)
time.sleep(5)

email_input = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/main/section[1]/div/div/form/div[1]/div[1]/div/div/input')))
email_input.send_keys(email)

password_input = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/main/section[1]/div/div/form/div[1]/div[2]/div/div/input')))
password_input.send_keys(password)

sign_in_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/section[1]/div/div/form/div[2]/button')))
sign_in_button.click()

time.sleep(2)
search_bar = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/header/div/div/div/div[1]/input')))
search_bar.send_keys(subject)
search_bar.send_keys(Keys.ENTER) 

all_people = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[1]/div/div[2]/a')))
all_people.click()

import re
for i in range(1,10):
    time.sleep(4)
    linkedin_url = None
    people = wait.until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/ul/li[{i}]/div/div/div[2]/div[1]/div[1]/div/span[1]/span/a')))
    people.click()
    time.sleep(3)

    try:
        full_name_element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[1]/h1')))
        full_name = full_name_element.text
        print("Name:", full_name)
        bio_element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[2]')))
        bio = bio_element.text
        print("Bio:", bio)
        contact_info_button = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[2]/span[2]/a')
        contact_info_button.click()
        
        try:
            time.sleep(3)
            contact_info = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[3]/section/div/section[1]/div/a')
            linkedin_url_text = contact_info.get_attribute("href")
            linkedin_url = re.search(r'(https?://www\.linkedin\.com/in/\S+)', linkedin_url_text).group()
            print("LinkedIn URL:", linkedin_url)
        except (NoSuchElementException, AttributeError):
            print("LinkedIn URL not found")
            
        data.append({
            'Name': full_name,
            'Bio': bio,
            'LinkedIn URL': linkedin_url})
    except NoSuchElementException:
        print("Contact info button not found")

    driver.back()
    time.sleep(1)
    driver.back()
    
df = pd.DataFrame(data)
current_dir = os.getcwd()
csv_path = os.path.join(current_dir, 'linkedin_profiles.csv')
df.to_csv(csv_path, index=False)
os.system(f'start "excel" "{csv_path}"')

Name: Mansi Chaudhary
Bio: RCI Registered (Rehabilitation Counsellor) | Mental Health Professional at Child Guidance Centre|
LinkedIn URL not found
Name: Preshit Gupta
Bio: Join the mental health revolution!
LinkedIn URL: https://www.linkedin.com/in/preshit-gupta-3903aa75
Name: Tarini Tandon
Bio: Attorney ⚖️ | Due-Diligence 👩‍💻 | Compliance 🙋‍♀️ | Contract Management 👩‍🎓 | Risk Strategist 🧏 | Fintech 👮‍♀️ | Mental Health Guide 🙇🏻
LinkedIn URL not found
Name: Abhishek Lakra
Bio: Mental Health Professional | Psychologist
LinkedIn URL: https://www.linkedin.com/in/-abhishek-lakra-
Name: Rohit Kumar Tiwari
Bio: Mental Health Practitioner
LinkedIn URL: https://www.linkedin.com/in/rohit-kumar-tiwari-a9b796145
Name: Sonam Nagpal
Bio: Digital & Growth Marketing | Sustainability | Mental Health
LinkedIn URL: https://www.linkedin.com/in/sonam-nagpal
Name: Josheka Chauhan
Bio: Counselling Psychologist | Mental Health Therapist
LinkedIn URL: https://www.linkedin.com/in/josheka
Name: Sachin Bajaj
Bi

0

In [7]:
import os
import yaml
import time
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd

def log_event(event):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("login_logs.txt", "a") as log_file:
        log_file.write(f"{event}, {timestamp}\n")

def read_yaml_file(file_path):
    with open(file_path) as f:
        data = yaml.safe_load(f)
        return data

def setup_driver(chromedriver_path):
    s = Service(chromedriver_path)
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()
    return driver

def login_to_linkedin(driver, page_url, email_input_xpath, password_input_xpath, sign_in_button_xpath, email, password):
    driver.get(page_url)
    time.sleep(1)
    driver.refresh()
    wait = WebDriverWait(driver, 10)
    time.sleep(3)

    log_event(f"Crawled {page_url}")

    email_input = wait.until(EC.presence_of_element_located((By.XPATH, email_input_xpath)))
    email_input.send_keys(email)

    password_input = wait.until(EC.presence_of_element_located((By.XPATH, password_input_xpath)))
    password_input.send_keys(password)

    sign_in_button = wait.until(EC.element_to_be_clickable((By.XPATH, sign_in_button_xpath)))
    sign_in_button.click()
    log_event("LOGGED IN with config.yml credentials")

    return wait

def extract_emails_from_text(text):
    email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    return re.findall(email_pattern, text)

def search_for_subject(wait, search_bar_xpath, subject, all_people_xpath):
    time.sleep(2)
    search_bar = wait.until(EC.presence_of_element_located((By.XPATH, search_bar_xpath)))
    search_bar.send_keys(subject)
    search_bar.send_keys(Keys.ENTER)
    log_event(f"SEARCHED FOR {subject}")

    try:
        all_people = wait.until(EC.element_to_be_clickable((By.XPATH, all_people_xpath)))
        all_people.click()
        time.sleep(3)
        log_event("Clicked on 'All' button")
    except TimeoutException as e:
        print("Error:", e)
        log_event("Error: TimeoutException - 'All' button not found")

def extract_contact_info(wait, linkedin_url_xpath):
    linkedin_url = None
    full_name = None
    email_id = None
    bio = None

    try:
        full_name_element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[1]/h1')))
        full_name = full_name_element.text
        print("Name:", full_name)
        log_event(f"CRAWLED through profile of {full_name}")
        
        bio_element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[2]')))
        bio = bio_element.text
        print("Bio:", bio)
        contact_info_button = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[2]/span[2]/a')
        contact_info_button.click()

        try:
            time.sleep(3)
            contact_info = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[3]/section/div/section[1]/div/a')
            linkedin_url_text = contact_info.get_attribute("href")
            linkedin_url = re.search(r'(https?://www\.linkedin\.com/in/\S+)', linkedin_url_text).group()
            print("LinkedIn URL:", linkedin_url)
        except (NoSuchElementException, AttributeError):
            print("LinkedIn URL not found")
             
        try:
            email_text = driver.page_source
            email_ids = extract_emails_from_text(email_text)
            if email_ids:
                email_id = email_ids[0]  # Assuming the first found email is the relevant one
                print("Email ID:", email_id)
            else:
                print("Email ID not found")
        except NoSuchElementException:
            print("Email ID not found")
            
    except NoSuchElementException:
        print("Contact info button not found")

    return full_name, bio, linkedin_url, email_id

if __name__ == '__main__':
    try:
        xpath_config_path = "xpath_config.yml"
        xpath_config = read_yaml_file(xpath_config_path)

        email_input_xpath = xpath_config["email_input"]
        password_input_xpath = xpath_config["password_input"]
        sign_in_button_xpath = xpath_config["sign_in_button"]
        search_bar_xpath = xpath_config["search_bar"]
        all_people_xpath = xpath_config["all_people"]
        linkedin_url_xpath = xpath_config.get("linkedin_url")  # Use get method to handle missing key

        config_path = "config.yml"
        config = read_yaml_file(config_path)

        email = config["username"]
        password = config["password"]
        subject = config["subject"]
        page_url = 'https://www.linkedin.com/?original_referer='

        driver = setup_driver("path_to_chromedriver")
        wait = login_to_linkedin(driver, page_url, email_input_xpath, password_input_xpath, sign_in_button_xpath, email, password)
        search_for_subject(wait, search_bar_xpath, subject, all_people_xpath)

        data = []
        for i in range(1, 10):
            time.sleep(4)
            
            people = wait.until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/ul/li[{i}]/div/div/div[2]/div[1]/div[1]/div/span[1]/span/a')))
            people.click()
            time.sleep(3)

            full_name, bio, linkedin_url, email_id = extract_contact_info(wait, linkedin_url_xpath)

            data.append({
                'Name': full_name,
                'Bio': bio,
                'Email': email_id,
                'LinkedIn URL': linkedin_url})

            driver.back()
            time.sleep(1)
            driver.back()

        df = pd.DataFrame(data)
        current_dir = os.getcwd()
        csv_path = os.path.join(current_dir, f'{subject}.csv')
        df.to_csv(csv_path, index=False)
        log_event("CREATED CSV")

        driver.quit()

        os.system(f'start "excel" "{csv_path}"')

    except Exception as e:
        print("An error occurred:", e)
        log_event(f"ERROR: {e}")


Name: Mansi Chaudhary
Bio: RCI Registered (Rehabilitation Counsellor) | Mental Health Professional at Child Guidance Centre|
LinkedIn URL not found
Email ID: entity-circle-pile-chat@2x.png
Name: Preshit Gupta
Bio: Join the mental health revolution!
LinkedIn URL: https://www.linkedin.com/in/preshit-gupta-3903aa75
Email ID: entity-circle-pile-chat@2x.png
Name: Ayushi Tangnoo
Bio: Mental Health Activist | An Engineer & Psychology Majors
LinkedIn URL: https://www.linkedin.com/in/ayushi-tangnoo7112
Email ID: entity-circle-pile-chat@2x.png
Name: Rohit Kumar Tiwari
Bio: Mental Health Practitioner
LinkedIn URL: https://www.linkedin.com/in/rohit-kumar-tiwari-a9b796145
Email ID: entity-circle-pile-chat@2x.png
An error occurred: Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0119A813+48355]
	(No symbol) [0x0112C4B1]
	(No symbol) [0x01035358]
	(No symbol) [0x010609A5]
	(No symbol) [0x01060B3B]
	(No symbol) [0x0108E232]
	(No symbol) [0x0107A784]
	(No symbol) [0x0108C922]
	(No symbol) [0x010

In [3]:
import os
import yaml
import time
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd

def log_event(event):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("login_logs.txt", "a") as log_file:
        log_file.write(f"{event}, {timestamp}\n")

def read_yaml_file(file_path):
    with open(file_path) as f:
        data = yaml.safe_load(f)
        return data

def setup_driver(chromedriver_path):
    s = Service(chromedriver_path)
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()
    return driver

def login_to_linkedin(driver, page_url, email_input_xpath, password_input_xpath, sign_in_button_xpath, email, password):
    driver.get(page_url)
    time.sleep(1)
    driver.refresh()
    wait = WebDriverWait(driver, 10)
    time.sleep(3)

    log_event(f"Crawled {page_url}")

    email_input = wait.until(EC.presence_of_element_located((By.XPATH, email_input_xpath)))
    email_input.send_keys(email)

    password_input = wait.until(EC.presence_of_element_located((By.XPATH, password_input_xpath)))
    password_input.send_keys(password)

    sign_in_button = wait.until(EC.element_to_be_clickable((By.XPATH, sign_in_button_xpath)))
    sign_in_button.click()
    log_event("LOGGED IN with config.yml credentials")

    return wait

def extract_emails_from_text(text):
    email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    return re.findall(email_pattern, text)

def search_for_subject(wait, search_bar_xpath, subject, all_people_xpath):
    time.sleep(2)
    search_bar = wait.until(EC.presence_of_element_located((By.XPATH, search_bar_xpath)))
    search_bar.send_keys(subject)
    search_bar.send_keys(Keys.ENTER)
    log_event(f"SEARCHED FOR {subject}")

    try:
        all_people = wait.until(EC.element_to_be_clickable((By.XPATH, all_people_xpath)))
        all_people.click()
        time.sleep(3)
        log_event("Clicked on 'All' button")
    except TimeoutException as e:
        print("Error:", e)
        log_event("Error: TimeoutException - 'All' button not found")

def extract_contact_info(wait, linkedin_url_xpath):
    linkedin_url = None
    full_name = None
    email_id = None
    bio = None

    try:
        full_name_element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[1]/h1')))
        full_name = full_name_element.text
        print("Name:", full_name)
        log_event(f"CRAWLED through profile of {full_name}")
        
        bio_element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[1]/div[2]')))
        bio = bio_element.text
        print("Bio:", bio)
        contact_info_button = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[2]/span[2]/a')
        contact_info_button.click()

        try:
            time.sleep(3)
            contact_info = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[3]/section/div/section[1]/div/a')
            linkedin_url_text = contact_info.get_attribute("href")
            linkedin_url = re.search(r'(https?://www\.linkedin\.com/in/\S+)', linkedin_url_text).group()
            print("LinkedIn URL:", linkedin_url)
        except (NoSuchElementException, AttributeError):
            print("LinkedIn URL not found")
             
        try:
            email_text = driver.page_source
            email_ids = extract_emails_from_text(email_text)
            if email_ids:
                email_id = email_ids[0]  # Assuming the first found email is the relevant one
                print("Email ID:", email_id)
            else:
                print("Email ID not found")
        except NoSuchElementException:
            print("Email ID not found")
            
    except NoSuchElementException:
        print("Contact info button not found")

    return full_name, bio, linkedin_url, email_id

if __name__ == '__main__':
    try:
        xpath_config_path = "xpath_config.yml"
        xpath_config = read_yaml_file(xpath_config_path)

        email_input_xpath = xpath_config["email_input"]
        password_input_xpath = xpath_config["password_input"]
        sign_in_button_xpath = xpath_config["sign_in_button"]
        search_bar_xpath = xpath_config["search_bar"]
        all_people_xpath = xpath_config["all_people"]
        linkedin_url_xpath = xpath_config.get("linkedin_url")  # Use get method to handle missing key

        config_path = "config.yml"
        config = read_yaml_file(config_path)

        email = config["username"]
        password = config["password"]
        subject = config["subject"]
        page_url = 'https://www.linkedin.com/?original_referer='

        driver = setup_driver("path_to_chromedriver")
        wait = login_to_linkedin(driver, page_url, email_input_xpath, password_input_xpath, sign_in_button_xpath, email, password)
        search_for_subject(wait, search_bar_xpath, subject, all_people_xpath)

        data = []
        for i in range(1, 10):
            time.sleep(4)
            
            people = wait.until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/ul/li[{i}]/div/div/div[2]/div[1]/div[1]/div/span[1]/span/a')))
            people.click()
            time.sleep(3)

            full_name, bio, linkedin_url, email_id = extract_contact_info(wait, linkedin_url_xpath)

            data.append({
                'Name': full_name,
                'Bio': bio,
                'Email': email_id,
                'LinkedIn URL': linkedin_url})

            driver.back()
            time.sleep(1)
            driver.back()

        df = pd.DataFrame(data)
        current_dir = os.getcwd()
        csv_path = os.path.join(current_dir, f'{subject}.csv')
        df.to_csv(csv_path, index=False)
        log_event("CREATED CSV")

        driver.quit()

        os.system(f'start "excel" "{csv_path}"')

    except Exception as e:
        print("An error occurred:", e)
        log_event(f"ERROR: {e}")


An error occurred: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location



In [2]:
pip install selenium


  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/dc/72/96b5afa16908f9abc7c24b70adfd3a46c9740eb728ddfeab28379e38eaf9/selenium-4.16.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/14/fb/9299cf74953f473a15accfdbe2c15218e766bae8c796f2567c83bae03e98/trio-0.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for outcome from https://files.pythonhosted.org/packages/55/8b/5ab7257531a5d830fc8000c476e63c935488d74609b50f9384a643ec0a62/outcome-1.3.0.post0-py2.py3-none-any.whl.metadata
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
   -------